In [1]:
#------------------import training data------and train classifier---------------------
#here we are training our classifer

In [2]:
# for demo purposes we have used tensorflow github data beforehand 

import json
f = open('real_data.json','r')
train = json.load(f) 
f.close() 

print(len(train["labels"]))

12331


In [3]:
#transform training data word count
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer()
X_train_tf=count_vect.fit_transform(train["titles"])
print(X_train_tf.shape)

(12331, 9399)


In [4]:
#apply inverse document frequency - reduced the weight of the words not necessary in categorizing data
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf=tfidf_transformer.fit_transform(X_train_tf)
print(X_train_tfidf.shape)

(12331, 9399)


In [5]:
#apply naive bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, train["labels"])

In [6]:
#------------------import data to be predict-------------and collect relevent data
import json

f = open('tensorflow_opened_issues.json','r')

issues_list = json.load(f) 

print(len(issues_list))

f.close() 

4030


In [7]:
#collect the issues titles, body and labels
data=[]
labels_list=[]
for i in range(len(issues_list)):
    single_data=[]
    issue=issues_list[i]
    issue_title=issue["title"]
    issue_body=issue["body"]
    
    single_data=single_data+[issue_title,issue_body];
    
    data=data+[single_data];
    
    issue_labels_list = issue["labels"]
    single_label_list=[]
    for j in range(len(issue_labels_list)):
        label=issue_labels_list[j]
        label_name=label["name"]
        single_label_list=single_label_list+[label_name]
    labels_list=labels_list+[single_label_list]

In [8]:
print(len(data),len(labels_list))

4030 4030


In [9]:
# collect all the titles of issues
issues_titles=[]
for i in range(len(data)):
    issues_titles=issues_titles+[data[i][0]]
print(len(issues_titles))

4030


In [10]:
#variables to find how many times we are predicting each category
real_cat0=0
real_cat1=0
real_cat2=0
real_cat3=0
real_cat4=0
real_cat5=0

pred_cat0=0
pred_cat1=0
pred_cat2=0
pred_cat3=0
pred_cat4=0
pred_cat5=0

def my_func(a):
    global pred_cat0,pred_cat1,pred_cat2,pred_cat3,pred_cat4,pred_cat5
    if(a==0):
        pred_cat0=pred_cat0+1
    elif(a==1):
        pred_cat1=pred_cat1+1
    elif(a==2):
        pred_cat2=pred_cat2+1
    elif(a==3):
        pred_cat3=pred_cat3+1
    elif(a==4):
        pred_cat4=pred_cat4+1
    elif(a==5):
        pred_cat5=pred_cat5+1



In [11]:
#------------------separate all the issues by their labels--------or predict labels for them---------classification
#now we are predicting labels for all open issues
import re

issue_type=[]
issue_num=[]

predicted_issues=0
non_predicted_issues=0

#for analysis purpose
labeled_issues=[]
labeled_issues_num=[]

for i in range(len(labels_list)):
    lb=labels_list[i]
    l=len(lb)
    if(l>0):
        itr=0
        for j in range(len(lb)):
            if(re.search("docs-bug", lb[j])):
                issue_type=issue_type+[['docs-bug']]
                issue_num=issue_num+[0]
                real_cat0=real_cat0+1
                non_predicted_issues=non_predicted_issues+1
                labeled_issues=labeled_issues+[issues_titles[i]]
                labeled_issues_num=labeled_issues_num+[0]
                break
            elif(re.search("bug", lb[j])):
                issue_type=issue_type+[['bug']]
                issue_num=issue_num+[1]
                real_cat1=real_cat1+1
                non_predicted_issues=non_predicted_issues+1
                labeled_issues=labeled_issues+[issues_titles[i]]
                labeled_issues_num=labeled_issues_num+[1]
                break
            elif(re.search("feature", lb[j])):
                issue_type=issue_type+[['feature']]
                issue_num=issue_num+[2]
                real_cat2=real_cat2+1
                non_predicted_issues=non_predicted_issues+1
                labeled_issues=labeled_issues+[issues_titles[i]]
                labeled_issues_num=labeled_issues_num+[2]
                break
            elif(re.search("support", lb[j])):
                issue_type=issue_type+[['support']]
                issue_num=issue_num+[3]
                real_cat3=real_cat3+1
                non_predicted_issues=non_predicted_issues+1
                labeled_issues=labeled_issues+[issues_titles[i]]
                labeled_issues_num=labeled_issues_num+[3]
                break
            elif(re.search("performance", lb[j])):
                issue_type=issue_type+[['performance']]
                issue_num=issue_num+[4]
                real_cat4=real_cat4+1
                non_predicted_issues=non_predicted_issues+1
                labeled_issues=labeled_issues+[issues_titles[i]]
                labeled_issues_num=labeled_issues_num+[4]
                break
            elif(re.search("build/install", lb[j])):
                issue_type=issue_type+[['build/install']]
                issue_num=issue_num+[5]
                real_cat5=real_cat5+1
                non_predicted_issues=non_predicted_issues+1
                labeled_issues=labeled_issues+[issues_titles[i]]
                labeled_issues_num=labeled_issues_num+[5]
                break
            itr=itr+1
        if(itr==l):
            #prediction of unrecognized issue
            issue_type=issue_type+[['unrecognized']] #keep it as unrecognized
            
            #transform test data
            X_test_tf=count_vect.transform([issues_titles[i]])
            X_test_tfidf=tfidf_transformer.transform(X_test_tf)
            predicted = clf.predict(X_test_tfidf)
            
            issue_num=issue_num+[predicted[0]]
            predicted_issues=predicted_issues+1
            my_func(predicted[0])
    else:
        #prediction of unrecognized issue
        issue_type=issue_type+[['unrecognized']] #keep it as unrecognized
        
        #transform test data
        X_test_tf=count_vect.transform([issues_titles[i]])
        X_test_tfidf=tfidf_transformer.transform(X_test_tf)
        predicted = clf.predict(X_test_tfidf)
        
        issue_num=issue_num+[predicted[0]]
        predicted_issues=predicted_issues+1
        my_func(predicted[0])
print(len(issue_type))
print(len(issue_num))

4030
4030


In [12]:
print("labeled")
print(len(labeled_issues))
print(len(labeled_issues_num))

labeled
3318
3318


In [13]:
#count labeled and unlabeled issues-------------------------------note- we have predicted only for unlabeled issues --- 712/4030
print("no of total issues: ",len(issue_type))
print("no of predicted issues: ",predicted_issues)
print("no of non-predicted issues: ",non_predicted_issues)

no of total issues:  4030
no of predicted issues:  712
no of non-predicted issues:  3318


In [14]:
#check how mant times we got predicted label in each category
#-----------say in the docs-bug class we got 175 issues in which 2 are predicted
print("no of predicted issues:     ",pred_cat0,pred_cat1,pred_cat2,pred_cat3,pred_cat4,pred_cat5)
print("no of non-predicted issues: ",real_cat0,real_cat1,real_cat2,real_cat3,real_cat4,real_cat5)
print("no of predicted issues:     ",pred_cat0+pred_cat1+pred_cat2+pred_cat3+pred_cat4+pred_cat5)
print("no of non-predicted issues: ",real_cat0+real_cat1+real_cat2+real_cat3+real_cat4+real_cat5)

no of predicted issues:      2 453 3 160 0 94
no of non-predicted issues:  173 1387 773 455 158 372
no of predicted issues:      712
no of non-predicted issues:  3318


In [15]:
#issues and their type
for i in range(len(issue_type)):
    print(issue_type[i],issue_num[i])

['build/install'] 5
['unrecognized'] 1
['performance'] 4
['bug'] 1
['build/install'] 5
['unrecognized'] 1
['unrecognized'] 1
['support'] 3
['docs-bug'] 0
['feature'] 2
['bug'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['bug'] 1
['build/install'] 5
['build/install'] 5
['unrecognized'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 5
['unrecognized'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 1
['unrecognized'] 1
['performance'] 4
['support'] 3
['support'] 3
['support'] 3
['bug'] 1
['docs-bug'] 0
['unrecognized'] 3
['support'] 3
['support'] 3
['bug'] 1
['support'] 3
['build/install'] 5
['support'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 1
['support'] 3
['unrecognized'] 3
['bug'] 1
['bug'] 1
['unrecognized'] 1
['unrecognized'] 1
['support'] 3
['support'] 3
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 5
['performance'] 4
['unrecognized'] 1
['support'] 3
['unrecognized'] 1
['support'] 3
['support'] 3
['unrecognized'] 1
['bug'] 1

['bug'] 1
['support'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 1
['feature'] 2
['support'] 3
['bug'] 1
['build/install'] 5
['performance'] 4
['bug'] 1
['bug'] 1
['feature'] 2
['support'] 3
['feature'] 2
['feature'] 2
['feature'] 2
['bug'] 1
['feature'] 2
['bug'] 1
['unrecognized'] 1
['support'] 3
['bug'] 1
['support'] 3
['unrecognized'] 5
['build/install'] 5
['feature'] 2
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['support'] 3
['performance'] 4
['bug'] 1
['unrecognized'] 1
['feature'] 2
['feature'] 2
['bug'] 1
['feature'] 2
['bug'] 1
['bug'] 1
['unrecognized'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['bug'] 1
['bug'] 1
['unrecognized'] 1
['build/install'] 5
['support'] 3
['build/install'] 5
['feature'] 2
['bug'] 1
['bug'] 1
['unrecognized'] 3
['bug'] 1
['bug'] 1
['feature'] 2
['bug'] 1
['support'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['build/install'] 5
['feature'] 2
['bug'] 1
['bug'] 1
['build/install'] 5
['unrecog

['bug'] 1
['feature'] 2
['unrecognized'] 5
['bug'] 1
['support'] 3
['feature'] 2
['feature'] 2
['bug'] 1
['docs-bug'] 0
['bug'] 1
['support'] 3
['feature'] 2
['support'] 3
['support'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['support'] 3
['docs-bug'] 0
['bug'] 1
['bug'] 1
['bug'] 1
['build/install'] 5
['feature'] 2
['bug'] 1
['docs-bug'] 0
['bug'] 1
['bug'] 1
['feature'] 2
['feature'] 2
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['performance'] 4
['bug'] 1
['build/install'] 5
['build/install'] 5
['performance'] 4
['support'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['build/install'] 5
['bug'] 1
['performance'] 4
['bug'] 1
['feature'] 2
['support'] 3
['bug'] 1
['docs-bug'] 0
['feature'] 2
['bug'] 1
['feature'] 2
['feature'] 2
['bug'] 1
['feature'] 2
['unrecognized'] 1
['bug'] 1
['feature'] 2
['feature'] 2
['support'] 3
['bug'] 1
['bug'] 1
['build/install'] 5
['build/install'] 5
['feature'] 2
['unrecognized'] 1
['bug'] 1
['bug'] 1
['support'] 3
['support'] 3
['unrecognized'] 1
['feature'] 

['unrecognized'] 1
['docs-bug'] 0
['feature'] 2
['feature'] 2
['support'] 3
['performance'] 4
['support'] 3
['feature'] 2
['bug'] 1
['feature'] 2
['docs-bug'] 0
['bug'] 1
['bug'] 1
['support'] 3
['feature'] 2
['support'] 3
['build/install'] 5
['bug'] 1
['feature'] 2
['docs-bug'] 0
['build/install'] 5
['feature'] 2
['docs-bug'] 0
['docs-bug'] 0
['docs-bug'] 0
['unrecognized'] 1
['build/install'] 5
['feature'] 2
['bug'] 1
['support'] 3
['feature'] 2
['bug'] 1
['feature'] 2
['feature'] 2
['feature'] 2
['feature'] 2
['feature'] 2
['bug'] 1
['feature'] 2
['bug'] 1
['build/install'] 5
['build/install'] 5
['unrecognized'] 5
['performance'] 4
['bug'] 1
['build/install'] 5
['unrecognized'] 3
['feature'] 2
['build/install'] 5
['support'] 3
['docs-bug'] 0
['build/install'] 5
['docs-bug'] 0
['bug'] 1
['bug'] 1
['support'] 3
['support'] 3
['bug'] 1
['bug'] 1
['support'] 3
['support'] 3
['feature'] 2
['support'] 3
['bug'] 1
['bug'] 1
['docs-bug'] 0
['support'] 3
['bug'] 1
['build/install'] 5
['suppo

['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 5
['build/install'] 5
['unrecognized'] 3
['unrecognized'] 3
['unrecognized'] 1
['bug'] 1
['unrecognized'] 1
['unrecognized'] 3
['unrecognized'] 5
['unrecognized'] 3
['build/install'] 5
['unrecognized'] 1
['support'] 3
['unrecognized'] 1
['unrecognized'] 3
['unrecognized'] 3
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 3
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 3
['unrecognized'] 3
['unrecognized'] 3
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['bug'] 1
['unrecognized'] 1
['bug'] 1
['unrecognized'] 3
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 1
['bug'] 1
['unrecognized'] 5
['support'] 3
['unrecognized'] 1
['unrecognized'] 1
['bug'] 1
['unrecognized'] 3
['feature'] 2
['unrecognized'] 1
['unrecognized'] 5
['unrecognized'] 1
['

In [16]:
# total number of issues in each category
list1=issue_num
cat0=0
cat1=0
cat2=0
cat3=0
cat4=0
cat5=0
for i in range(len(list1)):
    if(list1[i]==0):
        cat0=cat0+1
    elif(list1[i]==1):
        cat1=cat1+1
    elif(list1[i]==2):
        cat2=cat2+1
    elif(list1[i]==3):
        cat3=cat3+1
    elif(list1[i]==4):
        cat4=cat4+1
    elif(list1[i]==5):
        cat5=cat5+1
print(cat0,cat1,cat2,cat3,cat4,cat5)

175 1840 776 615 158 466


In [17]:
#------------------analyse data---------by labeled issues and their type----------------
for i in range(len(labeled_issues)):
    print()
    print(labeled_issues[i],"  ",labeled_issues_num[i])


install_pip_packages.sh specifies incompatible version combo for pylint and astroid    5

`tf.distribute.MirroredStrategy(),` but getting unbalance GPU usage    4

Error when using quantization aware training with custom activation function    1

Tensorflow failed to build due to error C2039 C3861 on windows with MSVC    5

rtx3090 tf2.4.1 error with single machine multi-card    3

https://www.tensorflow.org/guide/estimators - HTTP Error 404: Page not Found    0

Cosine annealing learning rate scheduler with minimum learning rate boundary    2

GPUOptions C++ free(): invalid pointer    1

Untraced function warning and Model parsing failure for Keras TCN Regressor (TF Lite)    1

Building tensorflow_cc target (cpu-only) from scratch with xla failing (TF1.15.5)    5

flatbuffers download:    5

micro: LOG_SOFTMAX PR2    1

micro: LOG_SOFTMAX PR1    1

Saving model in TF 2.4    1

micro: LOG_SOFTMAX PR3-5    1

"import tensorflow" configures Python logging if tensorboard is not installed


TFlite Poor Performance on Pixel 3 CPU when channel not multiple of 4    4

Weight Normalization layer doesn't work with mixed precision    1

Incorrect conda dependency information for tensorflow-gpu    5

Bug in Transpose Convolution Padding    3

 When I use AndroidStudio to import Image Segementation TensorsorflowLite model, there is a problem...    3

java.lang.NoClassDefFoundError: Failed resolution of: Lorg/tensorflow/lite/support/image/ColorSpaceType    3

micro: port operator DIV kernel from lite with test    1

TF 2.4.0 crashes on startup with IndexError assuming that sys.argv[0] exists when it may be hosted by C++ (regression from 2.3.1)    5

How to access operations of the tf graph from a keras model? (for tf 2.4.0)    1

cannot execute exe of tensorflow py compiled with pyinstaller    5

Unrecognized DataType enum value 68 and crash in malloc    4

TF 2.4: Metrics not recognized as stateful in ProgbarLogger callback    1

TensorflowLite Android OpenCL delegate may produc

Docs: provide CUDA/CuDNN/TensorRT version information for PyPI packages    2

Masking support for multiple values.    2

ParseTensor (tf.io.parse_tensor) is not vectorized - Vectorizing via tf.vectorized_map uses while_loop    4

Multiple instances of custom metrics don't behave as expected with multiple output model when evaluating.    1

More details and examples for tf.image.generate_bounding_box_proposals    0

Remove CMSIS Core include from all Makefiles    1

Add Ubuntu 20.04 installation instructions    5

InvalidArgumentError with `Conv1D` which has the same `dilation_rate` as the batch size, using `tf.keras.Model.fit`, on TPU    1

[TransformGraph & TF2 SavedModel] - Missing API    1

XLA Compilation does not work with Embeddings Layer    1

Wrong maximum value passed in the positional encoding for the Transformer model    0

tf.keras.layers.experimental.preprocessing.CategoryEncoding: is it possible to apply it to a nD tensor?    2

IteratorGetNext: unsupported op: No registe

Bug when serializing optimizer with `tf.keras.utils.serialize_keras_object` but works fine with `tf.keras.optimizers.serialize`    1

TFLu wrong predictions for optimized model    1

issue with parallelizing dataset    3

[TF 2.3.0] [Intel MKL] OMP Threads are created with variable number when we compile with XLA flag ON    1

Post-training quantization    1

Dot model generation not working when input_shape is not specified (Maybe just missing error message)     1

CUDA illegal error access error when running distributed mixed precision    1

Add "align_corners" and "half_pixel_centers" as argument to tf.image.resize     2

[TFTRT] Possible issue with Conv2D converter without `is_dynamic_op= True`    1

LSTM loss and accuracy values differ greatly between GPU and CPU implementation    1

tf.io.gfile.GFile truncates gzipped files loaded from Google Cloud buckets    1

Cannot test op using mutlithreading    1

Could not detect op for DEPTHWISE_CONV_2D    1

tf.image.random_flip_left_rig

TFLite Failed to build gpu delegate on Linux    5

Update NCCL version to the latest release v2.6.4-1    5

Performance issue with cross compiled benchmark_model and label_img    4

TFLite Conversion: ValueError: This converter can only convert a single ConcreteFunction. Converting multiple functions is under development.     3

Keras model compiled with custom loss raises "object has no attribute '__name__'" error.    1

Cannot convert lstm with "stateful=True" to tflite    1

tf.lookup.StaticHashTable: Cannot convert a Tensor of dtype resource to a NumPy array    1

How to limit GPU memory usage in java?    2

Master build failed with user local specific GCC    5

ERROR: Did not get operators, tensors, or buffers in subgraph 0.    1

Embedding layer ~10x slower in tf.function    4

TF2.2.0rc2: exception if shape of input to tf.linag.set_diag not fully specified     1

Add a Mutable Hash Table of Ragged Tensors     2

estimator.train_and_evaluate() sometimes throws errors during train

GPU memory on Bahdanau Attention    4

tensorflow 2.0.0 crashes with protobuf 3.10.1 on macOS    5

Can anyone build a optimized tensorflow-gpu-v2.0 based on cuda10.0 and cudnn7.6(centos7) for me?     5

Deadlock on recursive tf.function-decorated function    1

TFLite GPU Delegate on iOS: failed assertion `Cannot create a buffer of zero length.    1

Error: Not in GZIP format    1

`files_io.get_matching_files` fails for valid filenames that contain globs    1

non_max_suppression is full of bugs!    1

Error while trying to serilize Image Captioning keras model '_UserObject' object is not callable'    3

Bijectors are orders of magnitude slower in tf2.1 autograph distributed mirrored single-gpu mode    4

Cannot build gl_delegate of android    1

tensorflowlib for 2.0    5

Enable SO_REUSEPORT option in tensorflow training server    2

TensorflowLite gives wrong results when use GPU delegate    1

Tensorflow Python modules import the same module under multiple different names    3

Y

[TF 2.0 API Docs] tf.queue.FIFOQueue    0

Deep Learning VM  train failed    1

[2.0] SparseTensor shape becomes none after AutoGraph    1

Unicode op tests fail on s390x with "Could not create converter for input encoding: SHIFT-JIS" error    1

TFLite GPU supported ops not working    1

Support RaggedTensors in sequence feature columns     2

Error trying tensorflow litem operations are not supported by GPU delegate    1

Documenting the Ref keyword from operations input/output types    0

What does GradientTape.gradient(target, ...) when target is a list of tensors and not a single tensor ?    0

CMake build fails v1.13.1 fatal error: tensorflow/stream_executor/dnn.pb.h: No such file or directory  #include "tensorflow/stream_executor/dnn.pb.h"    5

Support the __cuda_array_interface__ protocol    2

Group convolutions UnimplementedError for nightly build    2

[Translation] Translation for Korean    0

Adding multiclass support to BoostedTreesClassifier    2

Tensorflow lite elemen

meta_graph_transform does not work with default checkpoint_path    1

ValueError: No variables to save    1

Block Matrix / Block Diagonal Matrix    2

 Predict fuel efficiency: regression example should not normalize one-hot values    0

[tf.keras.layers.LSTM] reset_state performance issue    1

tf.data equivalent for tf.estimator.inputs.numpy_input_fn    2

DistributionStrategy and Keras models: support for sample_weight_mode    2

bazel error for saved_model_cli    1

Hard arbitrary limit on Saved model size    1

Add tf.metrics.std or tf.metrics.var    2

tf.image (CLAHE) contrast limited adaptive histogram equalization.    2

[Feature Request] Batch size scheduler to speed up training    2

Error while running sample cpp program in tensorflow    5

Bug when passing multiple output of Lambda Layer to the Model API    1

Add an ability to terminate tf.data.choose_from_datasets/sample_from_datasets early    2

SparseTensor file format    2

CMake build Tensorflow C++ on Windows 10 Er

In [18]:
#prediction for labeled issues
predicted_num=[]
for i in range(len(labeled_issues)):
    #transform test data
    X_test_tf=count_vect.transform([labeled_issues[i]])
    X_test_tfidf=tfidf_transformer.transform(X_test_tf)
    predicted = clf.predict(X_test_tfidf)
        
    predicted_num=predicted_num+[predicted[0]]
print(len(predicted_num))

3318


In [20]:
#print labeled issues and their type and predicted values
#here we are checking their real label with predicted label for analyzing
for i in range(len(labeled_issues)):
    
    
    
    print(labeled_issues[i],"  ",labeled_issues_num[i],"  ",predicted_num[i])

install_pip_packages.sh specifies incompatible version combo for pylint and astroid    5    1
`tf.distribute.MirroredStrategy(),` but getting unbalance GPU usage    4    1
Error when using quantization aware training with custom activation function    1    1
Tensorflow failed to build due to error C2039 C3861 on windows with MSVC    5    5
rtx3090 tf2.4.1 error with single machine multi-card    3    3
https://www.tensorflow.org/guide/estimators - HTTP Error 404: Page not Found    0    0
Cosine annealing learning rate scheduler with minimum learning rate boundary    2    3
GPUOptions C++ free(): invalid pointer    1    1
Untraced function warning and Model parsing failure for Keras TCN Regressor (TF Lite)    1    1
Building tensorflow_cc target (cpu-only) from scratch with xla failing (TF1.15.5)    5    5
flatbuffers download:    5    5
micro: LOG_SOFTMAX PR2    1    1
micro: LOG_SOFTMAX PR1    1    1
Saving model in TF 2.4    1    1
micro: LOG_SOFTMAX PR3-5    1    1
"import tensorflow

Broken link in doc    0    0
Race condition in port picker leads to test failures    1    1
_pywrap_tensorflow_internal.pyd is to big after PyInstaller pack    5    5
tf.keras.preprocessing.image_dataset_from_directory includes hidden dirs as classes    1    1
Misleading warning when loading weights from hdf5 format weights    1    1
Tensorboard not loading scalars with reload arrow button    1    1
tflite model can not inference when use tensorflow op    1    3
macOS Target //tensorflow/tools/pip_package:build_pip_package failed to build in debug mode    5    5
How the quantization on BERT    1    3
tf.Variable throws TypeError on conversion to typed numpy ndarray    1    1
Auto-generated cortex-m0 hello world make project failing to link    1    5
 An error occurs when inferring by loaded pretrained model    3    1
tf.nn.convolution crashes Floating-point exception when filters has 0 in shape    1    1
Add Windows build to nightly libtensorflow C packages    1    5
Extracting item fr

Training takes forever when applying distribute strategy    4    1
Incorrect Cupti dll name on W10    5    5
Memory leak in TF 2.3.1 when using Multi-worker training    4    1
How can I use micro_mutable_op_resolver.h instead of all_ops_resolver.h to run TensorFlow Lite for Microcontrollers exapmle?    3    3
Bad performance of tf.data.Dataset API when dealing with windowed inputs / timeseries problems / 3D arrays    4    1
BUG: Keras SaveModel does not properly save optimizer state    1    1
tf.linalg.pinv also for complex matrices    2    1
Version requirements for Python dependencies too strict    2    5
test with random fail on graph mode    1    1
Tensorflow 2+ framework class exception    1    1
Move EthosU custom op out of AllOpsResolver    3    1
ambiguous template instantiation error with Eigen code    5    5
cuDNN LSTMs not utilized after loading a model using `tf.saved_model.load`    1    1
Tensorflow 2.3.1 is not compatible with cuda 10.2 on Windows    5    5
Different beha

How to save tf.data.Dataset object?    2    3
Failed to convert SparseTensor to Tensor    1    3
bug when using "num_parallel_calls" when mapping dataset to tfa function    1    1
nan gradient issue    1    1
Uninitialized memory access of per-channel params    1    1
NMS support in XLA    1    1
TensorArray issues in XLA    1    1
control flow support issue in TensorFlow XLA    1    3
Tensorflow model dump in c++ during inference    2    3
ValueError: Unable to save the object (Saving a Keras.model)    1    1
Group Conv3D malloc abort trap 6     1    1
org.tensorflow:tensorflow:jar:2.3.0 not found via Maven    5    5
tflite delagte to nn accelerator    3    1
which tensorflow version starts to support XLA compiler?    5    5
libtensorflowlite XNNPACK support beyond f32    2    5
tensorflowlite v2.3 + XNNPACK run into error {ModifyGraphWithDelegate is disallowed}    1    3
STM32F407G Build Error - AddBuiltin() mismatch in arguments    1    1
Error on doing regression with weighted time

Loss function not accessible when overriding optimizer_v2.OptimizerV2    1    1
random in tf.data.Dataset.map is not random if not coming from tensorflow    1    1
keras.models.clone_model ignores input_tensors for functional models    1    1
If TMP is not set, then the build system defaults to the wrong path    5    5
Dynamical Tensor (and EagerTensor) slice assignment    2    1
tf.signal.rfft(2d/3d) documentation refers `Tcomplex` and `input` as arguments.    0    1
tf.signal.irfft(2d/3d) documentation refers `input` and `Treal` as arguments.    0    1
Trying to dynamically change weighted connections between training examples    2    3
The GPUs hang when split a log_prob and gradient computation across a number of GPU devices    1    1
JAVA DataType support float16?    2    3
TFLite General OpenGL delegate     2    1
TF 2.2 GPU memory usage regression on model.predict() with big inputs    4    1
Feature request: facemesh example for tensorflow lite    2    3
[TF2.2] Build libtensorf

tf.numpy_function with lambda function is much slower starting from TF 1.15    1    1
Excessive memory consumption and preparation runtime of tf.keras.backend.max in custom layer with masking    1    1
Configurable attribute "srcs" doesn't match this configuration    5    3
No gradient defined for op    1    1
map_fn + @tf.function + tf.nn.conv2d throws error when strides to conv2d are supplied in the elems argument for map_fn    1    1
Resource temporarily unavailable, ENOMEM on CPU with memory rlimit    4    3
Increase BiasGrad size:  "BiasGrad requires tensor size <= int32 max"    2    1
Python crashes when computing max/min of a complex tensor    1    1
InaccessibleTensorError in graph mode with for loop itteration (different sized tensors) ( tf.image.resize)    1    1
AttributeError: module 'tensorboard' has no attribute 'lazy'    5    3
Encountered unresolved custom op: BatchMatMulV2.    1    3
tf.Graph.get_tensor_by_name does not work as expected in TF2    1    1
TensorFlow 2.1.

about tf.gradients() API    2    3
Fix behavior difference between tf.io.GFile and python file for utf8    1    1
tf.compat.v1.wrap_function throws error when creating variable in tf.distribute scope    1    1
Tracing of first batch does not collect compute events    1    3
TimeseriesGenerator for labeled time-series such as sensor data    2    3
Support for Mediapipe tflite ops.    2    3
TF2.0 for compute capability 3.0    2    1
Finish making strided_slice equivalent to numpy's strided_slice    2    3
tf.lite operators missing for `tf.nn.ctc_loss`: `Empty`, `InplaceAdd` and `While`    1    1
Using GPU for Tflite on Android    3    3
internal compiler error: in assign_temp, at function.c:968    5    1
tf2.0 takes more than twice memory than 1.14    4    1
tfliteGpuDelegate Invoke write to buffer failed source data is larger than buffer    3    3
Docker Image with a pre-built Tensorflow-gpu with a compute capability 3.0 GPUs    5    5
[TF 2.0] Custom objects    3    3
Cannot train can

error: default initialization of an object of const type 'const Subgraph::Identity' without a user-provided default constructor    5    1
Signature defs lost when calling saved_model_cli convert    1    1
ExtractImagePatches Op Request    2    3
catch keyboard interrupt in tf.function    2    1
TF Lite select tf ops example linker error: libtensorflow-lite.a missing tflite subgraph    1    1
TF Lite can't run example project of select tensorflow operators on iOS    1    3
TensorFlow 1.14 changes Keras callback order relative to model build    1    1
[tensorflow/docs]Translate variables.md from English to Chinese.    0    1
TFLite only slightly faster with GPU on Helio P22    1    3
Assigning to ressource variables with varying shapes violates the shape information of the tensor of which is assigned from.    1    1
use GradientTape to compute gradient cost all memory    3    3
defun + random + addition = explode    1    1
How to build Tensorflow C++ API with Visual Studio 2017    5    5

2.0 Reference Models: Keras Application Set (TPU)    2    1
2.0 Reference Models: Transformer (TPU with dist strat and Keras)    2    1
2.0 Reference Models: NMT Model (TPU with dist strat and Keras)    2    1
tf.linalg.solve segfaults on invalid matrix dimensions    1    1
Add integer data types to IsTrainable for use with custom gradients    2    1
how to export tensorrt int8  model with savedmodel format after calibration？    3    1
Feature: TF Serving compatibility with TF 2.0.    2    1
Feature: Unify argument names (~70).    2    1
Feature: TensorBoard compatibility with TF 2.0.    2    1
Library Conversion: Tensor2Tensor    2    3
2.0 Reference Models: NCF Model (TPU with Keras)    2    1
2.0 Reference Models: Keras Application Set (1 GPU)    2    1
[BUG] gradients of `tf.cond` with float64 breaks with ResourceVariable    1    1
Data augmentation on 4-D or 5-D data    2    1
windows c_api error with tensorflow.dll    5    5
Build issue of "cannot find symbol class Fill where T i

Distributed TensorFlow without shared directory    2    1
Include grpc_tensorflow_std_server in Docker image    2    5
Include netstat in the tensorflow docker container    2    5
Graph Transform Tool unable to build in TF source r1.5?    5    5
Faster R-CNN: too many resources requested for launch    1    3
Segmentation fault when running optimization step with 3d convolution    1    1
  tf.estimator.train_and_evaluate list  of eval_spec    2    1
Lack of Complex64 support for Java API    2    1
Allow tensorflow/tensorflow/workspace.bzl to customize dependencies    2    5
[Tracking bug] Building Tensorflow with Clang on Windows    3    5
cmake CUDA include-path whitespaces not supported    3    5
Feature Request: Sparse Cholesky decomposition    2    1
tensorflow/go for Windows?    3    5
Reinitializing an iterator throws an OutOfRangeError when using a MonitoredSession with NanTensorHook    2    1
Numerous ::`anonymous namespace':: Linking errors in Tensorflow 1.3.1 Windows build wit

In [21]:
#count how many times both the values are same
count=0
for i in range(len(labeled_issues)):
    if(labeled_issues_num[i]==predicted_num[i]):
        count=count+1
print(count,len(predicted_num))


1571 3318


In [22]:
print(train["cat"])

['docs-bug', 'bug', 'feature', 'support', 'performance', 'build/install']


In [23]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
print("Accuracy:\n",accuracy_score(labeled_issues_num,predicted_num))
print()
print(metrics.classification_report(labeled_issues_num,predicted_num,target_names=train["cat"]))
print()
print("confusion matrix:\n",metrics.confusion_matrix(labeled_issues_num,predicted_num))


Accuracy:
 0.4734779987944545

               precision    recall  f1-score   support

     docs-bug       0.82      0.08      0.15       173
          bug       0.51      0.78      0.62      1387
      feature       0.92      0.01      0.03       773
      support       0.24      0.39      0.30       455
  performance       0.00      0.00      0.00       158
build/install       0.66      0.77      0.71       372

     accuracy                           0.47      3318
    macro avg       0.53      0.34      0.30      3318
 weighted avg       0.58      0.47      0.39      3318


confusion matrix:
 [[  14  119    0   36    0    4]
 [   0 1085    0  257    0   45]
 [   3  543   11  188    0   28]
 [   0  210    1  176    0   68]
 [   0  118    0   39    0    1]
 [   0   56    0   31    0  285]]


C:\Users\rohit shakya\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
#-------give weightage to each category  ["docs-bug","bug","feature","support","performance","build/install"]-----prioritization
#and here we are giving weitage to each issue based on their class, based on author_association, based on assigned or not, and baded on amout of conversation
w=[]
for i in range(len(issue_num)):
    if(issue_num[i]==0):
        w=w+[0.2]
    elif(issue_num[i]==1):
        w=w+[0.5]
    elif(issue_num[i]==2):
        w=w+[0.4]
    elif(issue_num[i]==3):
        w=w+[0.0]
    elif(issue_num[i]==4):
        w=w+[0.1]
    elif(issue_num[i]==5):
        w=w+[0.3]
print(len(issue_num),len(w))

4030 4030


In [25]:
#check
for i in range(len(w)):
    print(w[i])

0.3
0.5
0.1
0.5
0.3
0.5
0.5
0.0
0.2
0.4
0.5
0.5
0.5
0.5
0.5
0.3
0.3
0.5
0.5
0.5
0.5
0.5
0.5
0.3
0.5
0.5
0.5
0.5
0.5
0.1
0.0
0.0
0.0
0.5
0.2
0.0
0.0
0.0
0.5
0.0
0.3
0.0
0.5
0.5
0.5
0.5
0.0
0.0
0.5
0.5
0.5
0.5
0.0
0.0
0.5
0.5
0.3
0.1
0.5
0.0
0.5
0.0
0.0
0.5
0.5
0.4
0.4
0.4
0.5
0.3
0.5
0.5
0.4
0.0
0.5
0.0
0.5
0.5
0.3
0.5
0.5
0.3
0.3
0.5
0.5
0.5
0.5
0.0
0.4
0.5
0.0
0.5
0.5
0.1
0.5
0.5
0.4
0.5
0.0
0.3
0.0
0.4
0.4
0.1
0.5
0.5
0.3
0.3
0.0
0.0
0.5
0.0
0.5
0.1
0.0
0.4
0.1
0.5
0.5
0.0
0.5
0.5
0.3
0.5
0.5
0.3
0.0
0.5
0.3
0.0
0.5
0.3
0.5
0.5
0.3
0.5
0.5
0.4
0.5
0.2
0.4
0.3
0.5
0.5
0.3
0.4
0.0
0.5
0.5
0.5
0.4
0.4
0.0
0.5
0.5
0.5
0.5
0.5
0.5
0.0
0.3
0.3
0.3
0.5
0.5
0.0
0.5
0.5
0.3
0.4
0.5
0.0
0.0
0.5
0.4
0.0
0.3
0.5
0.5
0.1
0.5
0.3
0.0
0.4
0.5
0.0
0.5
0.3
0.5
0.5
0.3
0.4
0.5
0.1
0.5
0.5
0.3
0.5
0.4
0.5
0.5
0.5
0.5
0.0
0.0
0.5
0.0
0.0
0.1
0.5
0.0
0.5
0.3
0.5
0.0
0.5
0.5
0.5
0.0
0.3
0.5
0.3
0.3
0.4
0.3
0.5
0.0
0.0
0.5
0.3
0.3
0.5
0.0
0.4
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.0
0.5
0.3
0.5
0.5
0.5
0.1


0.5
0.5
0.5
0.0
0.4
0.5
0.4
0.3
0.0
0.5
0.5
0.0
0.0
0.4
0.0
0.4
0.3
0.4
0.3
0.4
0.0
0.0
0.0
0.0
0.5
0.0
0.4
0.5
0.5
0.3
0.4
0.5
0.5
0.5
0.0
0.5
0.5
0.5
0.5
0.5
0.5
0.4
0.4
0.5
0.5
0.0
0.4
0.5
0.3
0.4
0.5
0.4
0.0
0.3
0.0
0.0
0.5
0.0
0.5
0.4
0.0
0.5
0.0
0.0
0.0
0.0
0.3
0.5
0.0
0.4
0.5
0.4
0.0
0.5
0.4
0.0
0.3
0.4
0.5
0.5
0.3
0.0
0.3
0.5
0.3
0.4
0.0
0.4
0.3
0.0
0.0
0.5
0.0
0.4
0.4
0.0
0.4
0.4
0.4
0.5
0.4
0.4
0.4
0.5
0.4
0.3
0.4
0.4
0.4
0.0
0.5
0.4
0.5
0.4
0.0
0.5
0.4
0.0
0.0
0.5
0.5
0.4
0.4
0.4
0.5
0.0
0.4
0.5
0.5
0.0
0.4
0.4
0.0
0.4
0.4
0.0
0.4
0.3
0.4
0.4
0.0
0.4
0.4
0.5
0.4
0.0
0.4
0.4
0.5
0.5
0.5
0.5
0.5
0.4
0.3
0.3
0.5
0.5
0.5
0.2
0.3
0.5
0.4
0.4
0.4
0.5
0.4
0.4
0.5
0.4
0.4
0.4
0.4
0.5
0.4
0.4
0.4
0.4
0.4
0.4
0.4
0.4
0.4
0.4
0.5
0.4
0.4
0.0
0.3
0.5
0.5
0.4
0.4
0.5
0.5
0.4
0.4
0.4
0.4
0.4
0.5
0.4
0.3
0.5
0.4
0.4
0.4
0.4
0.5
0.4
0.5
0.3
0.3
0.0
0.1
0.4
0.0
0.4
0.4
0.4
0.4
0.4
0.4
0.4
0.4
0.4
0.4
0.5
0.5
0.4
0.4
0.4
0.3
0.4
0.4
0.4
0.5


In [26]:
#------------------------------prioritize issues based on amount of conversation------------------------------------
number_of_comments=[]
for i in range(len(issues_list)):
    print(issues_list[i]["comments"])

1
0
2
1
1
1
0
2
1
0
1
0
0
0
2
1
1
0
0
0
1
2
4
0
3
1
7
2
1
2
1
2
2
1
9
2
1
4
1
2
2
1
20
6
4
4
1
1
3
4
2
1
1
5
0
1
4
2
0
1
4
1
3
2
4
3
4
0
2
1
3
2
2
0
3
3
3
4
1
2
5
3
0
1
0
1
2
3
0
4
2
11
1
2
4
1
0
3
5
1
2
0
3
3
1
2
4
1
7
0
5
0
3
1
11
0
1
2
10
1
2
4
2
1
2
2
6
8
4
2
2
5
3
1
2
4
0
0
1
0
0
0
2
4
3
1
7
5
2
1
1
0
1
1
7
4
10
2
2
3
4
2
2
6
10
4
4
1
5
2
0
2
1
2
4
4
12
0
2
6
8
1
2
2
3
2
2
5
3
3
11
2
14
7
4
1
2
1
3
0
2
3
1
4
2
11
4
4
2
1
1
5
0
10
6
3
1
0
5
7
7
6
2
5
0
7
3
6
1
4
7
5
8
0
3
2
0
6
2
1
2
1
3
5
2
0
3
0
0
6
7
5
3
6
3
1
1
0
0
1
0
0
1
2
6
6
7
2
3
4
4
0
1
4
1
5
1
1
1
11
0
0
1
0
1
3
21
0
7
8
11
2
2
1
3
0
10
2
4
1
1
1
2
1
2
1
1
1
3
1
1
2
1
1
4
4
2
1
2
1
1
2
2
1
1
1
5
1
3
6
10
3
5
4
3
5
6
1
6
0
2
8
1
2
5
1
4
1
1
0
0
9
5
11
2
2
4
2
2
8
5
3
0
3
7
7
0
0
0
2
0
2
5
4
2
13
3
4
7
0
8
0
1
1
3
2
4
0
0
4
2
2
2
3
3
0
1
2
5
1
2
4
0
4
0
0
15
0
0
9
10
2
0
3
6
8
1
3
0
5
1
3
4
13
3
1
3
3
1
4
3
1
2
4
2
2
2
6
0
2
3
9
0
4
1
1
1
1
20
4
1
9
6
5
3
0
4
9
8
0
1
2
4
3
7
1
8
3
3
0
2
2
3
2
17
2
1
2
6
6
5
3
5
2
0
2
3
0
6

In [27]:
#-----------------------------add weightage by number of commits (amount of conversation)-----------------------------
for i in range(len(issue_num)):
    w[i]=w[i]+0.01*issues_list[i]["comments"]
print(len(issue_num),len(w))

4030 4030


In [28]:
#check
for i in range(len(w)):
    print(w[i])

0.31
0.5
0.12000000000000001
0.51
0.31
0.51
0.5
0.02
0.21000000000000002
0.4
0.51
0.5
0.5
0.5
0.52
0.31
0.31
0.5
0.5
0.5
0.51
0.52
0.54
0.3
0.53
0.51
0.5700000000000001
0.52
0.51
0.12000000000000001
0.01
0.02
0.02
0.51
0.29000000000000004
0.02
0.01
0.04
0.51
0.02
0.32
0.01
0.7
0.56
0.54
0.54
0.01
0.01
0.53
0.54
0.52
0.51
0.01
0.05
0.5
0.51
0.33999999999999997
0.12000000000000001
0.5
0.01
0.54
0.01
0.03
0.52
0.54
0.43000000000000005
0.44
0.4
0.52
0.31
0.53
0.52
0.42000000000000004
0.0
0.53
0.03
0.53
0.54
0.31
0.52
0.55
0.32999999999999996
0.3
0.51
0.5
0.51
0.52
0.03
0.4
0.54
0.02
0.61
0.51
0.12000000000000001
0.54
0.51
0.4
0.53
0.05
0.31
0.02
0.4
0.43000000000000005
0.13
0.51
0.52
0.33999999999999997
0.31
0.07
0.0
0.55
0.0
0.53
0.11
0.11
0.4
0.11
0.52
0.6
0.01
0.52
0.54
0.32
0.51
0.52
0.32
0.06
0.58
0.33999999999999997
0.02
0.52
0.35
0.53
0.51
0.32
0.54
0.5
0.4
0.51
0.2
0.4
0.3
0.52
0.54
0.32999999999999996
0.41000000000000003
0.07
0.55
0.52
0.51
0.41000000000000003
0.4
0.01
0.51
0.5700

0.54
0.44
0.05
0.53
0.56
0.48000000000000004
0.52
0.07
0.42000000000000004
0.52
0.3
0.46
0.41000000000000003
0.42000000000000004
0.43000000000000005
0.4
0.5700000000000001
0.56
0.07
0.55
0.42000000000000004
0.44
0.43000000000000005
0.14
0.44
0.56
0.52
0.5
0.52
0.59
0.5
0.58
0.5
0.52
0.31
0.44
0.23
0.52
0.5700000000000001
0.32
0.12000000000000001
0.13
0.4
0.07
0.51
0.4
0.14
0.4
0.55
0.31
0.53
0.65
0.53
0.51
0.03
0.5700000000000001
0.32999999999999996
0.54
0.44
0.4
0.3
0.51
0.41000000000000003
0.53
0.43000000000000005
0.71
0.4
0.53
0.52
0.26
0.42000000000000004
0.31
0.58
0.45
0.25
0.4
0.94
0.73
0.14
0.55
0.23
0.04
0.13
0.24000000000000002
0.45
0.52
0.03
0.53
0.5700000000000001
0.42000000000000004
0.43
0.51
0.01
0.43000000000000005
0.4
0.51
0.23
0.53
0.03
0.43000000000000005
0.05
0.02
0.5
0.51
0.7
0.0
0.2
0.61
0.55
0.58
0.3
0.4
0.56
0.23
0.5700000000000001
0.55
0.41000000000000003
0.44
0.52
0.51
0.54
0.4
0.13
0.54
0.38
0.3
0.19
0.08
0.5700000000000001
0.76
0.61
0.42000000000000004
0.37
0.

0.64
0.59
0.62
0.5700000000000001
0.49
0.5900000000000001
0.58
0.52
0.02
0.46
0.53
0.37
0.46
0.53
0.41000000000000003
0.01
0.44
0.21
0.1
0.5
0.06
0.59
0.42000000000000004
0.08
0.55
0.02
0.01
0.15
0.07
0.44
0.6
0.03
0.41000000000000003
0.8400000000000001
0.43000000000000005
0.16
0.79
0.46
0.03
0.43
0.46
0.52
0.81
0.44999999999999996
0.01
0.44999999999999996
0.62
0.42
0.44
0.08
0.47000000000000003
0.59
0.03
0.08
0.6
0.11
0.51
0.41000000000000003
0.05
0.74
0.45
0.65
0.53
0.48000000000000004
0.44
0.48000000000000004
0.79
0.44
0.45999999999999996
0.51
0.42000000000000004
0.43000000000000005
0.06
0.54
0.72
0.53
0.45
0.03
0.55
0.43000000000000005
0.43
0.04
0.73
0.56
0.42000000000000004
0.43000000000000005
0.71
0.5700000000000001
0.01
0.44
0.52
0.53
0.13
0.46
0.48000000000000004
0.12
0.46
0.46
0.05
0.73
0.32
0.5900000000000001
0.9500000000000001
0.01
0.44
0.46
0.55
0.5
0.04
0.49
0.49
0.56
0.67
0.61
0.64
0.58
0.45
0.31
0.38
0.59
0.56
0.58
0.23
0.31
0.55
0.42000000000000004
0.49
0.45
0.66
0.63
0

In [29]:
#-------------------------------prioritize issues based on the autor_association--------------------------------
for i in range(len(issues_list)):
    print(issues_list[i]["author_association"])

CONTRIBUTOR
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
CONTRIBUTOR
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
NONE
MEMBER
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
MEMBER
NONE
MEMBER
NONE
CONTRIBUTOR
MEMBER
NONE
MEMBER
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
MEMBER
NONE
NONE
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
MEMBER
CONTRIBUTOR
CONTRIBUTOR
MEMBER
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR


NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
MEMBER
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
MEMBER
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
MEMBER
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
MEMBER
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
NONE

CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
MEMBER
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
MEMBER
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
MEMBER
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE

In [30]:
#prioritize issues based on the autor_association    [OWNER, MEMBER, COLLABORATOR, CONTRIBUTOR, NONE, FIRST_TIMER, FIRST_TIME_CONTRIBUTOR]
for i in range(len(issue_num)):
    if(issues_list[i]["author_association"]=="OWNER"):
        w[i]=w[i]+0.004
    elif(issues_list[i]["author_association"]=="MEMBER"):
        w[i]=w[i]+0.003
    elif(issues_list[i]["author_association"]=="COLLABORATOR"):
        w[i]=w[i]+0.002
    elif(issues_list[i]["author_association"]=="CONTRIBUTOR"):
        w[i]=w[i]+0.001
print(len(issue_num),len(w))

4030 4030


In [31]:
#check
for i in range(len(w)):
    print(w[i])

0.311
0.5
0.12000000000000001
0.51
0.31
0.511
0.501
0.02
0.21000000000000002
0.4
0.51
0.501
0.501
0.5
0.52
0.31
0.31
0.5
0.501
0.501
0.511
0.52
0.541
0.301
0.53
0.51
0.5700000000000001
0.52
0.511
0.12000000000000001
0.01
0.02
0.02
0.51
0.29000000000000004
0.02
0.01
0.04
0.51
0.02
0.32
0.01
0.7
0.56
0.54
0.541
0.01
0.01
0.531
0.54
0.52
0.513
0.01
0.05
0.501
0.51
0.33999999999999997
0.12000000000000001
0.501
0.01
0.541
0.011
0.03
0.521
0.54
0.43000000000000005
0.443
0.4
0.523
0.31
0.531
0.523
0.42000000000000004
0.003
0.531
0.03
0.531
0.54
0.31
0.52
0.55
0.32999999999999996
0.3
0.511
0.501
0.51
0.52
0.03
0.4
0.54
0.02
0.611
0.511
0.12000000000000001
0.54
0.51
0.4
0.53
0.05
0.31
0.02
0.401
0.43300000000000005
0.13
0.51
0.521
0.33999999999999997
0.311
0.07
0.0
0.551
0.001
0.531
0.11
0.11
0.4
0.11
0.52
0.6
0.01
0.52
0.543
0.321
0.511
0.523
0.32
0.06
0.58
0.33999999999999997
0.02
0.521
0.35
0.53
0.51
0.32
0.54
0.501
0.401
0.51
0.2
0.4
0.3
0.52
0.54
0.33099999999999996
0.41000000000000003
0.0

0.42000000000000004
0.52
0.3
0.46
0.41300000000000003
0.42000000000000004
0.43000000000000005
0.4
0.5700000000000001
0.56
0.07
0.55
0.42100000000000004
0.441
0.43100000000000005
0.14
0.44
0.56
0.52
0.5
0.52
0.59
0.5
0.58
0.501
0.52
0.31
0.44
0.231
0.52
0.5700000000000001
0.323
0.12000000000000001
0.13
0.4
0.07100000000000001
0.51
0.4
0.14100000000000001
0.4
0.551
0.31
0.53
0.65
0.53
0.51
0.03
0.5700000000000001
0.32999999999999996
0.541
0.441
0.4
0.301
0.51
0.41000000000000003
0.53
0.43100000000000005
0.71
0.4
0.53
0.52
0.26
0.42000000000000004
0.31
0.58
0.45
0.251
0.401
0.941
0.73
0.14
0.55
0.23
0.04
0.131
0.24000000000000002
0.45
0.521
0.03
0.53
0.5700000000000001
0.42000000000000004
0.43
0.51
0.01
0.43000000000000005
0.401
0.51
0.23
0.531
0.03
0.43000000000000005
0.05
0.02
0.5
0.51
0.7
0.0
0.2
0.61
0.55
0.58
0.3
0.401
0.56
0.231
0.5700000000000001
0.551
0.41000000000000003
0.44
0.52
0.51
0.54
0.401
0.133
0.54
0.38
0.301
0.191
0.08
0.5700000000000001
0.76
0.61
0.42000000000000004
0.3

0.5700000000000001
0.56
0.531
0.5900000000000001
0.7
0.55
0.55
0.611
0.6799999999999999
0.53
0.03
0.07
0.59
0.06
0.531
0.53
0.51
0.24
0.6799999999999999
0.04
0.37
0.19
0.45
0.03
0.581
0.44
0.29
0.551
0.631
0.05
0.48000000000000004
0.611
0.5700000000000001
0.651
0.44
0.48100000000000004
0.29000000000000004
0.51
0.611
0.501
0.43000000000000005
0.04
0.64
0.03
0.45299999999999996
0.58
0.54
0.55
0.08
0.39
0.511
0.44
0.33999999999999997
0.52
0.53
0.35
0.47000000000000003
0.33999999999999997
0.441
0.15
0.58
0.611
0.56
0.32
0.491
0.611
0.5900000000000001
0.43000000000000005
0.581
0.44999999999999996
0.32999999999999996
0.46
0.351
0.63
0.56
0.411
0.541
0.43000000000000005
0.71
0.47100000000000003
0.041
0.08
0.6
0.43000000000000005
0.64
0.08
0.42000000000000004
0.05
0.48
0.06
0.44
0.62
0.5
0.61
0.5710000000000001
0.05
0.5
0.61
0.48000000000000004
0.34099999999999997
0.151
0.611
0.76
0.02
0.06
0.531
0.04
0.52
0.36
0.55
0.47
0.46
0.151
0.14100000000000001
0.06
0.07
0.55
0.09
0.551
0.701
0.521
0.46

In [32]:
#---------------------------prioritize issues based on the assignees (assigned or not)-----------------------------
for i in range(len(issues_list)):
    if(len(issues_list[i]["assignees"])):
        print(len(issues_list[i]["assignees"]))
    else:
        print(0)

1
1
1
2
1
0
0
1
1
1
1
0
0
0
3
1
1
1
2
2
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
2
1
1
1
1
2
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
2
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
2
1
1
1
1
1
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
2
1
2
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
2
1
2
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
4
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
4
4
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
2
1
1
1
1
1
4
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [33]:
#--------------------------------prioritize issues based on the assignees (assigned or not)---------------------------------
for i in range(len(issue_num)):
    if(len(issues_list[i]["assignees"])==0):
        w[i]=w[i]+0.0001
        
print(len(issue_num),len(w))

4030 4030


In [34]:
#check
for i in range(len(w)):
    print(w[i])

0.311
0.5
0.12000000000000001
0.51
0.31
0.5111
0.5011
0.02
0.21000000000000002
0.4
0.51
0.5011
0.5011
0.5001
0.52
0.31
0.31
0.5
0.501
0.501
0.511
0.52
0.541
0.301
0.53
0.51
0.5700000000000001
0.52
0.511
0.12000000000000001
0.01
0.02
0.02
0.51
0.29000000000000004
0.02
0.01
0.04
0.51
0.02
0.32
0.01
0.7
0.56
0.54
0.541
0.01
0.01
0.531
0.54
0.52
0.513
0.01
0.05
0.501
0.51
0.33999999999999997
0.12000000000000001
0.501
0.01
0.541
0.011
0.03
0.521
0.54
0.43000000000000005
0.443
0.4
0.523
0.31
0.531
0.523
0.42000000000000004
0.003
0.531
0.03
0.531
0.54
0.31
0.52
0.55
0.32999999999999996
0.3
0.511
0.501
0.51
0.52
0.03
0.4
0.54
0.02
0.611
0.511
0.12000000000000001
0.54
0.51
0.4
0.53
0.05
0.31
0.02
0.401
0.43300000000000005
0.13
0.51
0.521
0.33999999999999997
0.311
0.07
0.0
0.551
0.001
0.531
0.11
0.11
0.4
0.11
0.52
0.6
0.01
0.52
0.543
0.321
0.511
0.523
0.32
0.06
0.58
0.33999999999999997
0.02
0.521
0.35
0.53
0.51
0.32
0.54
0.501
0.401
0.51
0.2
0.4
0.3
0.52
0.54
0.33099999999999996
0.41000000000000

0.521
0.03
0.53
0.5700000000000001
0.42000000000000004
0.43
0.51
0.01
0.43000000000000005
0.401
0.51
0.23
0.531
0.03
0.43000000000000005
0.050100000000000006
0.02
0.5
0.51
0.7
0.0
0.2
0.61
0.55
0.58
0.3
0.401
0.56
0.231
0.5700000000000001
0.551
0.41000000000000003
0.44
0.52
0.51
0.54
0.401
0.133
0.54
0.38
0.301
0.191
0.08
0.5700000000000001
0.76
0.61
0.42000000000000004
0.371
0.51
0.14100000000000001
0.51
0.4
0.09
0.54
0.29000000000000004
0.4
0.61
0.443
0.77
0.56
0.41000000000000003
0.511
0.52
0.41000000000000003
0.451
0.0301
0.53
0.58
0.33999999999999997
0.3
0.4
0.551
0.54
0.66
0.0
0.06
0.6
0.41100000000000003
0.3
0.34099999999999997
0.24000000000000002
0.4
0.7
0.4
0.07
0.18
0.56
0.7
0.51
0.56
0.601
0.32
0.52
0.321
0.52
0.111
0.5
0.51
0.22
0.43000000000000005
0.1301
0.37
0.52
0.4
0.8
0.48
0.41000000000000003
0.351
0.56
0.65
0.41100000000000003
0.12000000000000001
0.51
0.511
0.403
0.521
0.56
0.4
0.521
0.03
0.52
0.26
0.62
0.401
0.55
0.4
0.5710000000000001
0.44
0.04
0.41000000000000003
0

0.07010000000000001
0.59
0.0601
0.531
0.5301
0.51
0.24
0.6799999999999999
0.04
0.3701
0.1901
0.4501
0.03
0.5811
0.4401
0.29
0.5511
0.631
0.050100000000000006
0.4801
0.6111
0.5701
0.651
0.4401
0.4811
0.29000000000000004
0.5101
0.611
0.5011
0.43010000000000004
0.040100000000000004
0.64
0.0301
0.45309999999999995
0.58
0.5401
0.55
0.0801
0.39
0.5111
0.4401
0.34009999999999996
0.52
0.53
0.35009999999999997
0.4701
0.34009999999999996
0.4411
0.15
0.5801
0.6111
0.5601
0.3201
0.491
0.611
0.5901000000000001
0.43010000000000004
0.581
0.44999999999999996
0.33009999999999995
0.46
0.35109999999999997
0.6301
0.56
0.41109999999999997
0.541
0.43010000000000004
0.7101
0.4711
0.041100000000000005
0.0801
0.6001
0.43010000000000004
0.6401
0.0801
0.42010000000000003
0.050100000000000006
0.48
0.0601
0.4401
0.6201
0.5001
0.6101
0.5711
0.050100000000000006
0.5001
0.6101
0.4801
0.34109999999999996
0.15109999999999998
0.611
0.7601
0.0201
0.0601
0.5311
0.040100000000000004
0.5201
0.3601
0.5501
0.47009999999999996

In [39]:
#-----------------------------print all the issues with their, type and weight
list_of_issues_titles=issues_titles
list_of_issues_types=issue_num
list_of_issues_weights=w

import pandas as pd

mydataset = {
  'title': list_of_issues_titles,
  'type': list_of_issues_types,
  'weight': list_of_issues_weights
}

myvar = pd.DataFrame(mydataset)
print(myvar)



                                                  title  type  weight
0     install_pip_packages.sh specifies incompatible...     5  0.3110
1     LSTMs with XNNPACK: ERROR: Attempting to use a...     1  0.5000
2     `tf.distribute.MirroredStrategy(),` but gettin...     4  0.1200
3     Error when using quantization aware training w...     1  0.5100
4     Tensorflow failed to build due to error C2039 ...     5  0.3100
...                                                 ...   ...     ...
4025         [clang+CUDA] No ZeroesLike[DT_BOOL] kernel     5  0.5511
4026  Feature request: other types of padding beside...     2  0.6211
4027  Warn when calling session.run() on a very larg...     2  0.5011
4028               Make TensorFlow compatible with PyPy     2  0.8201
4029                                     OpenCL support     1  5.9101

[4030 rows x 3 columns]


In [62]:
#--------------------------------------print all the issues in descending order----------------------------------------
#and this is our final output based on their weitage in decending order
myvar1=myvar.sort_values(by=['weight'], ascending=False)
print(myvar1)

                                                  title  type  weight
4029                                     OpenCL support     1  5.9101
3191  Could not create cudnn handle: CUDNN_STATUS_IN...     1  2.3100
3034  Tensorflow lite gpu delegate inference using o...     1  1.5200
3268  Feature Request - Deeplab TFLITE Android Appli...     2  1.2700
1777  Add __reduce_ex__ to Keras Model to enable cop...     1  1.2610
...                                                 ...   ...     ...
404                           Time issue in micro_speec     3  0.0000
961   How are batch gradients computed on embedding ...     3  0.0000
109          In-batch-negatives and distributed traning     3  0.0000
3069  Warning with custom categorical column with cu...     3  0.0000
1996  implicit conversion from 'int' to 'float' chan...     3  0.0000

[4030 rows x 3 columns]


In [66]:
new1=[]
new2=[]
for i in range(len(myvar1)):
    new1=new1+[myvar1.iloc[i][0]]
    new2=new2+[myvar1.iloc[i][1]]
print(len(new1),len(new2))

4030 4030


In [68]:
for i in range(len(myvar1)):
    print(new1[i],new2[i])

OpenCL support 1
Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR 1
Tensorflow lite gpu delegate inference using opengl and SSBO in android 1
Feature Request - Deeplab TFLITE Android Application 2
Add __reduce_ex__ to Keras Model to enable copy.deepcopy and pickle 1
Bidirectional LSTM fail on TF2.0 1
Address missing TensorFlow operations to TFLite: 2
failed to query event: CUDA_ERROR_LAUNCH_FAILED: unspecified launch failure tensorflow/stream_executor/dnn.cc:613] CUDNN_STATUS_INTERNAL_ERROR 3
Need tf.signal.rfft op in TFLite 2
C API Tensors 2
Linear RAM memory increase with Dataset and Estimator with epoch loops 1
using bazel to build tensorflow.dll 5
Tensorflow - Metal Support for Mac OS 2
TensorBoard callback without profile_batch setting cause Errors: CUPTI_ERROR_INSUFFICIENT_PRIVILEGES and CUPTI_ERROR_INVALID_PARAMETER 1
[TFLite] Fully quantize the uint8 concatenation kernel to make it pure integer 1
tf.keras.Sequential() fails 1
tf.function-decorated function tried to cr

tensorflow lite tests on x86 1
null pointer dereference Error in TF2.3.0 with runforMultipleInputOutput 1
saved_model_cli convert hub module to tensorRT model 1
Missing GPU op for zeros_like for RaggedTensorVariant, error occurs when Ragged Tensor fed thru tf.map_fn 1
Tensorflow and flask has an error loading model. 1
un-deprecate `write_grads` for `fit` 2
Allow Keras Callbacks to access predictions on_batch_end, on_epoch_end 1
Dataset.from_generator() returns a Dataset which freezes execution upon iteration 1
Bad input tensor parameters in model 1
tf.train.Checkpoint complain that tf.train.ExponentialMovingAverage instance is not a trackable object 1
Duplicate variables for models whose layers share weights 1
Android model personalization crash with keras model 1
tfio.audio.AudioIOTensor doesn't prepare data in time in loop. 1
Spurious tf.function retracing warnings, when developing Keras layer in colab 1
keras.layers.experimental.preprocessing.Discretization fails for mixed precision

PosixFileSystem::CreateDir should create directory respecting umask 1
micro: port op ADD_N from lite 1
add MNIST training to `mnist_grad_test` 1
micro: port op LEAKY_RELU from lite 1
micro: port op FLOOR_DIV from lite 1
Remove CMSIS Core include from all Makefiles 1
micro: port op DEPTH_TO_SPACE from lite 1
InfeedEnqueueTuple placed on TPU device blocks indefinitely 1
TF Unit test flaky in haswell/broadwell machines 1
micro: port op FILL from lite 1
Add missing closing backtick 1
Remove unnecessary model parsing from keras.model.load_model 1
Go implementation does not provide SummaryImage/SummaryAudio 1
micro: port op DIV from lite 1
[Intel MKL] Enabling Dequantize op with native format 1
Missing CI for OPTIMIZED_KERNEL_DIR=cmsis_nn with MVEI extension 1
[Intel MKL] Fix shape inference for Quantized{Conv2D/DepthwiseConv2D}-like operations 1
Embedding visualization in tf.keras / TensorFlow 2.0 1
Eager: Random seeds 1
micro: port op FLOOR_MOD from lite 1
Saved_model should be able to sav

[Feature] Support RISCV with tfcompile --target_tuple 5
./configure [--help|-h] does not work 5
warnings (please do not import '@grpc//third_party/nanopb:pb_common.c' directly ; depends on deprecated target ; ...) during build from source 5
Docker Image with a pre-built Tensorflow-gpu with a compute capability 3.0 GPUs 5
-D_GLIBCXX_USE_CXX11_ABI=1 increases a lot RAM usage 5
Segfault in custom op with TensorFlow 1.14, Python 3.6 and GCC 5. 5
An application using tensorflow_cc.dll crashes whenever the Tensorflow API is invoked. 5
Get path from environment CUDA_PATH  5
OUT_OF_MEMORY error on Mali GPU. 3
How to install Tensorflow on AIX7.2 server 5
macOS Target //tensorflow/tools/pip_package:build_pip_package failed to build in debug mode 5
Import Issue of cross-built tflite package 5
Documentation corresponding to Arguments for Pre-Trained Models inside <tf.keras.applications> is missing 0
Missing Documentation for compiling binaries in c++ 5
The current TensorFlow lite version cannot be

In [47]:
# import pandas library as pd
import pandas as pd
  
# List of Tuples
students = [('Ankit', 22, 'Up', 'Geu'),
           ('Ananya', 31, 'Delhi', 'Gehu'),
           ('Rahul', 16, 'Tokyo', 'Abes'),
           ('Simran', 41, 'Delhi', 'Gehu'),
           ('Shaurya', 33, 'Delhi', 'Geu'),
           ('Harshita', 35, 'Mumbai', 'Bhu' ),
           ('Priya', 35, 'Mp', 'Geu'),
           ('Priya', 34, 'Uk', 'Geu'),
           ('Jeet', 35, 'Guj', 'Gehu'),
           ('Ananya', 35, 'Up', 'Bhu')
            ]
  
# Create a DataFrame object from
# list of tuples with columns
# and indices.
details = pd.DataFrame(students, columns =['Name', 'Age',
                                           'Place', 'College'],
                        index =[ 'b', 'c', 'a', 'e', 'f', 
                                'g', 'i', 'j', 'k', 'd'])
  
# sort Dataframe rows based on "Name' 
# column in Descending Order
rslt_df = details.sort_values(by = 'Name', ascending = False)

print(rslt_df)
  
# show the resultant Dataframe
rslt_df.iloc[0]

       Name  Age   Place College
e    Simran   41   Delhi    Gehu
f   Shaurya   33   Delhi     Geu
a     Rahul   16   Tokyo    Abes
i     Priya   35      Mp     Geu
j     Priya   34      Uk     Geu
k      Jeet   35     Guj    Gehu
g  Harshita   35  Mumbai     Bhu
b     Ankit   22      Up     Geu
c    Ananya   31   Delhi    Gehu
d    Ananya   35      Up     Bhu


Name       Simran
Age            41
Place       Delhi
College      Gehu
Name: e, dtype: object

In [37]:
print(issues_list[4029]["comments"])   #there are so many comments on this issue, and its got heigher prority and it is still not resolved,

541


In [38]:
print(len(issues_list[4029]["labels"])) #there is only one label on this issue

1


In [39]:
print(issues_list[4029]["labels"][0]["name"]) #this label is not describing its class, so it is predicted as class 1
print(issues_list[4029]["labels"][0]["description"]) #------------Status - Contributions welcome----------------

stat:contributions welcome
Status - Contributions welcome


In [45]:
data=myvar.copy()

In [48]:
data.type=data.type.replace({1:"bug",0:"docs-bug",2:"feature",4:"performance",5:"build/install",3:"support"})

In [49]:
data.head()

,title,type,weight
0,install_pip_packages.sh specifies incompatible...,build/install,0.311
1,LSTMs with XNNPACK: ERROR: Attempting to use a...,bug,0.500
2,"`tf.distribute.MirroredStrategy(),` but gettin...",performance,0.120
3,Error when using quantization aware training w...,bug,0.510
4,Tensorflow failed to build due to error C2039 ...,build/install,0.310


In [50]:
output=data.sort_values(by=['weight'], ascending=False)

In [ ]:
output["priority_score"] = output["weight"]

In [69]:
output.head(50)

,title,type,weight
4029,OpenCL support,bug,5.9101
3191,Could not create cudnn handle: CUDNN_STATUS_IN...,bug,2.3100
3034,Tensorflow lite gpu delegate inference using o...,bug,1.5200
3268,Feature Request - Deeplab TFLITE Android Appli...,feature,1.2700
1777,Add __reduce_ex__ to Keras Model to enable cop...,bug,1.2610
2460,Bidirectional LSTM fail on TF2.0,bug,1.2600
3459,Address missing TensorFlow operations to TFLite:,feature,1.1710
1511,failed to query event: CUDA_ERROR_LAUNCH_FAILE...,support,1.0800
2928,Need tf.signal.rfft op in TFLite,feature,1.0210
3976,C API Tensors,feature,1.0011
